# Operation of a reservoir system
In this section the concepts of hedging rule, control curve and objective tradeoff in operation of a water reservoir system will be introduced by means of practical and interactive examples.

<left><img src="Images/Dam2.gif" width = "600px"><left>

## Simulation of a reservoir with a single natural inflow and a human demand outflow
The reservoir system used in this case study is a single reservoir system. The reservoir has a storage capacity of 150 Ml. The reservoir is required to meet both human demand and to make environmental compensation releases.

Assuming perfect 8-week inflow, temperature and demand forecasts, you should define an optimal weekly release policy.

<left> <img src="Images/system_representation1.png" width = "600px"><left>

### Inputs:

#### Variables:

$T$ = simulation time, in number of weeks

$I$ = natural reservoir inflows, in ML/week (Vector of length T)

$E$ = evaporation volume, in ML/week (Vector of length T)

$d$ = demand, in ML/week (Vector of length T)

#### Initial conditions:

$S_0$ = inital reservoir storage volume, in ML

#### Constraints:

$S_{max}$ = max reservoir storage volume, in ML

$S_{min}$ = min reservoir storage volume, in ML

$env_{min}$ = required environmental compensation flow, in ML/week

### Outputs:

$S$ = reservoir storage, in ML (Vector of length T)

##### Gravity outflows
$u$ = reservoir controlled releases for water supply, in ML/week (Vector of length T)

$w$ = spillways, in ML/week (Vector of length T)

$env$ = environmental compensation outflow, in ML/week

<left> <img src="Images/system_representation_IO1.png" width = "600px"><left>

### Simulation model of the system
First of all, we need to import the necessary libraries to run the model.

In [1]:
from bqplot import pyplot as plt
from bqplot import *
from bqplot.traits import *
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from Subroutines.Interactive_policy import Interactive_policy_single,Interactive_policy_double, Interactive_Pareto_front

#### Inputs definition
**1. Simulation time:** we introduce the number of weeks ($simtime$) that we would like to simulate. In this case 8 weeks.

In [2]:
simtime = 8

**2. Inputs:** forecast of weekly natural inflows ($I$), evaporation ($E$), temperature ($T$) and water demand ($d$). In this example we will manually define them by creating arrays of lenght equal to the simulation time (8).

In [3]:
### Forcing inputs ###
  # Natural inflows forecast
I = np.array([15,17,19,11,9,4,3,8]) # in ML/week
  # Temperature forecast
T = np.array([13,13,17,18,20,22,25,26]) # in degC
  # Evaporation forecast
E = T*0.1 # in ML/week. Evaporation is estimated as a function of the temperature in a simplistic way 
# (transformation factor = 0.1) just to reflect their interaction.
  # Water demand forecast
d = np.array([15]*simtime)*T/15 # in ML/week. Evaporation is estimated as a function of the temperature in a simplistic way 
# (transformation factor = 0.1) just to reflect their interaction.

Plot of the forecasted weekly inflows

In [4]:
# Axis characterisitcs
x_sc_1 = LinearScale();y_sc_1 = LinearScale(min=0,max=35)
x_ax_1 = Axis(label='week', scale=x_sc_1);y_ax_1 = Axis(label='ML/week', scale=y_sc_1, orientation='vertical')
# Bar plot
inflow_plot = plt.bar(np.arange(1,simtime+1),I,colors=['blue'],stroke = 'lightgray',scales={'x': x_sc_1, 'y': y_sc_1},
                      labels = ['inflow'], display_legend = True)
#Figure characteristics
fig_1a = plt.Figure(marks = [inflow_plot],title = 'Inflow forecast for the next 8 weeks', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1a])

In [5]:
# Bar plot (we use the same axis as the weekly inflows figure)
demand_plot   = plt.bar(np.arange(1,simtime+1),d,colors=['gray'],stroke = 'lightgray',opacities = [0.7]*simtime, 
                        labels = ['demand'], display_legend = True, 
                    stroke_width = 1,scales={'x': x_sc_1, 'y': y_sc_1})
#Figure characteristics
fig_1b = plt.Figure(marks = [demand_plot],title = 'Demand forecast for the next 8 weeks', axes=[x_ax_1, y_ax_1],
                    layout={'min_width': '1000px', 'max_height': '300px'}, legend_style = {'fill': 'white', 'opacity': 0.5})
widgets.VBox([fig_1b])

**3. Initial conditions:** initial reservoir storage volume.

In [6]:
### Initial conditions ###
  # Initial storage volume
S0 = 80 # in ML

**4. System constraints:** Now we define the constraints of the system such as the reservoir storage capacity or the minimum environmental flow. *But what is the environmental flow? [Learn more about the environmental flow.](https://www.youtube.com/watch?v=cbUrrYq9BmU)*

In [7]:
### Constraints ###
  # Max storage volume
Smax = 150 #  in ML
  # Min storage volume
Smin = 0 # in ML
  # Min environmental compensation flow
env_min = 2 # in ML/week

#### System model definition

In [8]:
def syst_sim(simtime,I,E,d,S0,Smax,env_min,u):

    # Declare output variables

    S = [0]*(simtime+1) # reservoir storage in ML

    w = [0]*(simtime) # spillage in ML

    env = [env_min]*(simtime) # environmental compensation flow
    
    S[0] = S0 # initial storage

    for t in range(simtime): # Loop for each time-step (week)

        # If at week t the inflow (I) is lower than the minimum environmental compensation (env_min), 
        # then the environmental compensation (env) = inflow (I)  
        if env_min >= I[t] :
            env[t] = I[t]
        # If the minimum environmental compensation is higher than the water resource available (S + I - E)
        # then the environmental compensation is equal to the higher value between 0 and the resource available
        if env_min >= S[t] + I[t] - E[t]:
            env[t] = max(0,S[t] + I[t] - E[t]) 
        # If the demand is higher than the water resource available (S + I - E - env)
        # then the release for water supply is equal to the higher value between 0 and the resource available            
        if d[t] >= S[t] + I[t] - E[t] - env[t]:
            u[t] = min(u[t],max(0,S[t] + I[t] - E[t] - env[t]))
        # The spillage is equal to the higher value between 0 and the resource available exceeding the reservoir capacity
        w[t] = max(0,S[t] + I[t] - u[t] - env[t] - E[t] - Smax)
        # The final storage (initial storage in the next step) is equal to the storage + inflow - outflows
        S[t+1] = S[t] + I[t] - u[t] - env[t]- E[t] - w[t]
        
    return S,env,w,u

## Water system operation: manual optimization approach
### Hedging rule

In the following example, given the forecasted water inflow ($I$) and demand ($d$) for the next 8 weeks, try to meet the weekly demand by increasing the weekly controlled releases ($u$) using the sliders. First try to meet the demand for the whole period. You will see that, it is possible to fully meet the demand for the first 7 weeks but not in the last week, causing severe deficits. 

In [9]:
# Interactive release policy
fig_2a,fig_2b,release1,release2,release3,release4,release5,release6,release7,release8 = Interactive_policy_single(simtime,I,E,d,S0,Smax,env_min, demand_plot)
HBox_layout = widgets.Layout(justify_content='center')
widgets.VBox([fig_2a,fig_2b,widgets.HBox([release1,release2,release3,release4,release5,release6,release7,release8],layout=HBox_layout)])

An operating rule that places more penalties on large deficits than small ones is called Hedging Rule. If the reservoir system manager tries to meet the demand fully during early months of a period, the manager may incur severe deficits on later periods. In order to prevent severe deficit in the later period, the manager can tolerate some deficit during release periods.

Now, try to find better solution by minimizing the severity of the water deficit, represented by the water supply deficit penalty ($sdpen$) and estimated by the following equation:

$$sdpen = \sum_{n=1}^{N} max(d(t)-u(t),0)^{2}$$

where $n$ is the week number and $N$ is the total number of weeks.

The record is **49, try to beat it!**

### From single to multi- objective optimization
A minimum storage threshold can be defined, below which, the quality of water or the reservoir recrational uses can be seriouly affected.
In our case we manually define the minimum storage as an array of length 8 weeks:

In [10]:
# Minimum storage threshold
ms = np.array([30,30,30,30,30,30,30,30,30]) # in ML

It can happen that due to specific conditions like low inflows, minimum requirements for demands etc. it is not possible to keep the storage volume over the minimum. 

Now in the following example try not only to meet the demand but also keep the storage level above the minimum threshold. The penalty applied when the storage lower than the minimum threshold ($lspen$) is estimated by:

$$lspen = \sum_{n=1}^{N} max ( rc(t) - S(t), 0)$$

Try to obtain the minimum supply deficit with no low storage penalty. The record is **305, try to beat it!**.

In [11]:
fig_3a,fig_3b,release1,release2,release3,release4,release5,release6,release7,release8 = Interactive_policy_double(simtime,I,E,d,S0,Smax,ms,env_min, demand_plot)
HBox_layout = widgets.Layout(justify_content='center')
widgets.VBox([fig_3a,fig_3b,widgets.HBox([release1,release2,release3,release4,release5,release6,release7,release8],layout=HBox_layout)])

## Water system operation: automatic optimization approach
Depending on the decision maker priority or the characteristics of our system one may select a different policy that for example, prioritizes the minimization of the supply deficit rather than the storage level. As we have seen, when we prioritize one objective the other one is usually deteriorated, this is what we call trade-off. In order to explore possible combinations of weekly releases and the obtained trade-off between objectives a more efficient procedure than manually is needed. By combining simulation and automatic optimization, we can obtain a representative set of optimal policies without having to consider all possible combinations of alternatives.
#### Algorithm to combine simulation and optimization

In [12]:
# Optimizer
from platypus import NSGAII, Problem, Real, Integer

def auto_optim(vars):
    release1 = vars[0]
    release2 = vars[1]
    release3 = vars[2]
    release4 = vars[3]
    release5 = vars[4]
    release6 = vars[5]
    release7 = vars[6]
    release8 = vars[7]

    u = [release1,release2,release3,release4,release5,release6,release7,release8]
    S,env,w,u = syst_sim(simtime,I,E,d,S0,Smax,env_min,u)
    
    sdpen = (np.sum((np.maximum(d-u,[0]*simtime))**2)).astype('int')
    lspen = (np.sum((np.maximum(ms-S,[0]*(simtime+1))))).astype('int')
    
    return [sdpen,lspen]

problem = Problem(simtime,2)
Real0 = Real(0, 40);Real1 = Real(0, 40);Real2 = Real(0, 40);Real3 = Real(0, 40);Real4 = Real(0, 40);Real5 = Real(0, 40);Real6 = Real(0, 40);Real7 = Real(0, 40)

problem.types[:] = [Real0] + [Real1] + [Real2] + [Real3] + [Real4] + [Real5] + [Real6] + [Real7] 
problem.function = auto_optim

population_size = 20
algorithm = NSGAII(problem,population_size)
algorithm.run(10000) # Number of iterations

results1_optim_relea = np.array([algorithm.result[i].objectives[0] for i in range(population_size)])
results2_optim_relea = np.array([algorithm.result[i].objectives[1] for i in range(population_size)])

solutions_optim_relea = [algorithm.result[i].variables[0:8] for i in range(population_size)]

#### Plot the Pareto front
In order to facilitate the decision-making process the Pareto front or tradeoff curve supplies a representation of a set of optimal solutions. The Pareto front gives full information on objective values and on objective tradeoffs, which inform how improving one objective is related to deteriorating the second one while moving along the tradeoff curve. Click on the Pareto front points and try to find the most balanced approach according to your criteria.

In [13]:
# Interactive Pareto front
fig_4a,fig_4b,fig_pf = Interactive_Pareto_front(simtime,I,E,d,S0,Smax,ms,env_min, demand_plot,solutions_optim_relea,results1_optim_relea,results2_optim_relea)
widgets.VBox([widgets.HBox([widgets.VBox([fig_4a,fig_4b]),fig_pf])])

#### Let's go to the next section!: [Reservoir operation under uncertainty](../3%20-%20Reservoir%20operation/Reservoir%20operation%20under%20uncertainty.ipynb)